In [24]:
import pandas as pd
import sys
sys.path.append(r'C:\Users\huzef\OneDrive\Documents\Projects\Projects\data_science_code')

from data_processor import DataProcessor
from classification_models import LogisticRegressionModel
from convert_categorical_variables import CategoricalEncoder
pd.set_option('display.max_columns', None)


In [25]:
data_processor = DataProcessor()
train_df = data_processor.read_csv(r'C:\Users\huzef\OneDrive\Documents\Projects\Projects\data\titanic\train.csv')
test_df = data_processor.read_csv(r'C:\Users\huzef\OneDrive\Documents\Projects\Projects\data\titanic\test.csv')
gender_submission = data_processor.read_csv(r'C:\Users\huzef\OneDrive\Documents\Projects\Projects\data\titanic\gender_submission.csv')

In [26]:
train_data, test_data = data_processor.train_test_split(train_df, test_size=0.2, random_state=42)
X_train, y_train = data_processor.split_features_target(train_data, 'Survived')
X_test, y_test = data_processor.split_features_target(test_data, 'Survived')

Data split into training and testing sets successfully.
Features and target variable split successfully.
Features and target variable split successfully.


# EDA/ Feature Engineering

In [31]:
def eda(df_1):
    df = df_1.copy()
    df.drop('PassengerId', axis=1, inplace=True)

    df['Surname'] = df['Name'].str.split(',').str[0]
    df['FirstName'] = df['Name'].str.split(',').str[1]
    df['Title'] = df['FirstName'].str.split('.').str[0]
    df.drop(['Name', 'FirstName', 'Surname'], axis=1, inplace=True)
    rare_titles = ['Rev', 'Dr', 'Major', 'Col', 'Mlle', 'Capt', 'Mme', 'the Countess', 'Lady', 'Sir', 'Jonkheer', 'Don']
    df['Title'] = df['Title'].apply(lambda x: 'Miss' if x.strip() == 'Ms' else 'Rare' if x.strip() in rare_titles else x.strip())

    df['Sex'] = df['Sex'].map({'male': 1, 'female': 0})

    df['Ticket_string'] = df['Ticket'].str.contains('[a-zA-Z]').astype(int)

    df['Cabin'] = df['Cabin'].fillna('U')

    most_common = df['Embarked'].mode()[0]
    df['Embarked'] = df['Embarked'].fillna(most_common)

    df['Age'] = df['Age'].fillna(df['Age'].median())

    df['Cabin_Letter'] = df['Cabin'].str.extract(r'([A-Za-z])')
    df['Cabin_Number'] = df['Cabin'].str.extract(r'(\d+)')
    df['Cabin_Number'] = df['Cabin_Number'].fillna(0)
    
    df = df[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked','Title','Ticket_string','Cabin_Letter','Cabin_Number']]

    encoder = CategoricalEncoder(df)
    df = encoder.one_hot_encoding(['Embarked', 'Title', 'Cabin_Letter'])
    df = df.drop(['Cabin_Letter_U'], axis=1)

    return df

X_train_filt = eda(X_train)
X_test_filt = eda(X_test)

In [32]:
X_train_filt.head(30)

,Pclass,Sex,Age,SibSp,Parch,Fare,Ticket_string,Cabin_Number,Embarked_C,Embarked_Q,Embarked_S,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Rare,Cabin_Letter_A,Cabin_Letter_B,Cabin_Letter_C,Cabin_Letter_D,Cabin_Letter_E,Cabin_Letter_F,Cabin_Letter_G,Cabin_Letter_T
331,1,1,45.5,0,0,28.5000,0,124,False,False,True,False,False,True,False,False,False,False,True,False,False,False,False,False
733,2,1,23.0,0,0,13.0000,0,0,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False
382,3,1,32.0,0,0,7.9250,1,0,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False
704,3,1,26.0,1,0,7.8542,0,0,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False
813,3,0,6.0,4,2,31.2750,0,0,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False
118,1,1,24.0,0,1,247.5208,1,58,True,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False
536,1,1,45.0,0,0,26.5500,0,38,False,False,True,False,False,False,False,True,False,True,False,False,False,False,False,False
361,2,1,29.0,1,0,27.7208,1,0,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False
29,3,1,28.0,0,0,7.8958,0,0,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False
55,1,1,28.0,0,0,35.5000,0,52,False,False,True,False,False,True,False,False,False,False,True,False,False,False,False,False


In [ ]:
X_train = encoder.one_hot_encoding(['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked'])
X_train.head()

In [ ]:
log_reg_model = LogisticRegressionModel()
log_reg_model.set_params(random_state=42, max_iter=1000) 
log_reg_model.train(X_train, y_train)
print(log_reg_model.get_coefficients())
print(log_reg_model.get_params())
